# [**Bart-large-cnn**](https://huggingface.co/facebook/bart-large-cnn?text=The+tower+is+324+metres+%281%2C063+ft%29+tall%2C+about+the+same+height+as+an+81-storey+building%2C+and+the+tallest+structure+in+Paris.+Its+base+is+square%2C+measuring+125+metres+%28410+ft%29+on+each+side.+During+its+construction%2C+the+Eiffel+Tower+surpassed+the+Washington+Monument+to+become+the+tallest+man-made+structure+in+the+world%2C+a+title+it+held+for+41+years+until+the+Chrysler+Building+in+New+York+City+was+finished+in+1930.+It+was+the+first+structure+to+reach+a+height+of+300+metres.+Due+to+the+addition+of+a+broadcasting+aerial+at+the+top+of+the+tower+in+1957%2C+it+is+now+taller+than+the+Chrysler+Building+by+5.2+metres+%2817+ft%29.+Excluding+transmitters%2C+the+Eiffel+Tower+is+the+second+tallest+free-standing+structure+in+France+after+the+Millau+Viaduct.)

## Dependecies

In [1]:
!pip install --user -U nltk
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 46.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 26.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 69.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 64.3 MB/s eta 0:00:00


In [1]:
import pandas as pd
import nltk
from nltk import tokenize
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import pipeline
import re

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Herman\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
# nltk.download('punkt')

## Read data

In [150]:
df = pd.read_csv('../data/summaries.csv')
df = df[df['juniorDescription'].notna() & df['seniorDescription'].notna()]
df.drop_duplicates(subset=["seniorDescription"], inplace=True)
df['seniorLength']  = df['seniorDescription'].str.len()

## Read data

In [151]:
df = df[df['juniorDescription'] != 'Proposal is too long']
df = df[df['seniorLength'] > 200]

In [152]:
display(df)

,id,title,daoId,juniorDescription,middleDescription,seniorDescription,startAt,endAt,author,createdAt,...,sum_word_density,text_word_density,sum_sent_density,text_sent_density,sum_punc_count,text_punc_count,text_stopw_count,sum_stopw_count,sum_stopw_density,text_stopw_density
0,1,[1IP-07] Integrate Balancer Boosted Pools in t...,1,The proposal is about integrating Balancer Boo...,This proposal calls for the integration of Bal...,Simple Summary. This proposal calls for the in...,1651554223000,1652159023000,0x824732d3f4eb94a20254cca9de10485ce445bb40,1657200979740,...,0.191837,0.195598,0.041667,0.049157,3,110,255,18,0.375000,0.358146
1,2,[1IP-06] Donation of 1inch DAO Treasury Funds ...,1,The 1inch DAO Treasury is proposing to donate ...,The 1inch DAO is proposing to donate 1 million...,Summary. 1inch Network was founded on core val...,1646760014000,1647364814000,0x824732d3f4eb94a20254cca9de10485ce445bb40,1657200981507,...,0.178295,0.191805,0.042553,0.053864,4,77,150,14,0.297872,0.351288
2,3,[Temperature Check] Should the Uniswap communi...,2,The Protocol Guild is a council of Ethereum pr...,"The authors are proposing that 500,000 UNI (ab...","Authors: [Trent]([link]) (PG Member), [Tim]([l...",1654176425000,1654437600000,0x4c0a466df0628fe8699051b3ac6506653191cc21,1657200982022,...,0.205036,0.178934,0.034483,0.048402,10,317,384,20,0.344828,0.350685
3,4,Aave V3 Harmony - Freeze Reserves,3,The proposal is about freezing all reserves on...,"The Aave DAO Community, through the governance...",title: Aave V3 Harmony - Freeze Reserves. stat...,1657807792000,1658138400000,0xd2362dbb5aa708bc454ce5c3f11050c016764fa6,1658239945117,...,0.193966,0.194444,0.043478,0.058394,5,34,100,17,0.369565,0.364964
4,5,Revised ARC: Add 1INCH as collateral,3,The proposal seeks to add 1inch Networks 1INCH...,1inch Network is a decentralized set of protoc...,Summary. 1inch is a network of decentralized p...,1657638000000,1657983600000,0xc290cfb8d020c0615e9c63036f4319cc41717e68,1658239945766,...,0.219048,0.192007,0.042553,0.070053,3,430,588,19,0.404255,0.314439
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
678,805,[BIP-237] Enable USH - ETH 50/50 Gauge with 10...,12,This proposal is about adding a gauge to the n...,This proposal is about adding a gauge to the n...,[PR with Payload]([link]) Summary: Proposal to...,1680796800000,1681142400000,0x9f74662aD05840Ba35d111930501c617920dD68e,1680720307179,...,0.183575,0.190299,0.034783,0.042445,9,250,419,46,0.400000,0.355688
679,806,[BIP-239] Enable bb-a-USD gauge [Ethereum],12,This passage is describing a proposal to add a...,This passage is describing a proposal to add a...,[PR with Payload]([link]) Summary This is a pr...,1680796800000,1681142400000,0x9f74662aD05840Ba35d111930501c617920dD68e,1680720307225,...,0.203390,0.200301,0.041237,0.067416,14,79,93,38,0.391753,0.348315
680,807,[BIP-240] Enable rETH/bb-a-wETH Gauge [Arbitrum],12,This proposal is for a new pool on Arbitrum cr...,This proposal is for a new pool on Arbitrum cr...,[link] Summary: > This pool uses the Composabl...,1680796800000,1681142400000,0x512fce9B07Ce64590849115EE6B32fd40eC0f5F3,1680776108847,...,0.205882,0.184189,0.039370,0.050773,18,243,137,46,0.362205,0.302428
681,808,[BIP-241] Aave v3 Migration,12,This proposal is about transitioning from the ...,This proposal is about transitioning from the ...,[link] Motivation. > With the Aave v3 linear p...,1680796800000,1681142400000,0x512fce9B07Ce64590849115EE6B32fd40eC0f5F3,1680777004583,...,0.213793,0.180944,0.042553,0.039171,9,286,139,38,0.404255,0.320276


## Split text to chunks <= 1024 tokens

In [6]:
checkpoint = "facebook/bart-large-cnn"

tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)
print(f'Model max length {tokenizer.model_max_length}')

Model max length 1024


In [118]:
def split_text(text, limit, sep=" "):
  sentences_l0 = tokenize.sent_tokenize(text)
  sentences_l1 = []
  delimiter = '|'
  for sent in sentences_l0:
    sent_splited = sent.split(delimiter)
    if len(sent_splited) == 1:
      sentences_l1.append(sent_splited[0])
    else:
      my_list = [x+delimiter for x in sent_splited if len(sent_splited)>1]
      for i in my_list:
        sentences_l1.append(i)

  res, part, others = [], sentences_l1[0], sentences_l1[1:]
  for word in others:
    if len(sep)+len(word) > limit-len(part):
        res.append(part)
        part = word
    else:
        part += sep+word
  if part:
    res.append(part)
  return res

In [156]:
df['seniorSplit'] = df['seniorDescription'].map(lambda x: split_text(x, limit=1024)).copy()

In [157]:
long_speech_split = (
    pd.DataFrame(df['seniorSplit'].explode())
    .stack()
    .reset_index()
    .rename(columns={0: "Sections"})
    .drop(["level_1"], axis=1)
)

In [158]:
display(long_speech_split)

,level_0,Sections
0,0,Simple Summary. This proposal calls for the in...
1,0,These pools were previously discussed with the...
2,0,In the near future we expect to see meta pools...
3,0,Support for the mentioned pools will unlock $3...
4,0,The technical specifics of this integration wi...
...,...,...
2755,682,[[1IP-14] Update 1inch DAO Voting Process to I...
2756,682,"Currently, all v1INCH, no matter where it is a..."
2757,682,New scheme = 17K votes Example 3 Address: [hex...
2758,682,Change the proposal validation strategy to req...


### Length checks

In [164]:
sections_list = long_speech_split['Sections'].tolist()
lens_sections_tokenized = [len(tokenizer.tokenize(c)) for c in sections_list]

In [165]:
long_speech_split['SectionsLength']  = lens_sections_tokenized
display(long_speech_split[long_speech_split['SectionsLength'] > 1024])

,level_0,Sections,SectionsLength


In [172]:
display(long_speech_split.sort_values(by='SectionsLength'))

,level_0,Sections,SectionsLength
2421,592,Copyright Copyright and related rights waived ...,10
2545,625,In reference to this discussion: [link] Backgr...,11
1987,495,Consolidate the utility of the Opium governanc...,11
1399,362,data(bytes) : '[hexadecimals]',12
2473,602,Consolidate the utility of the Opium governanc...,12
...,...,...,...
934,230,The BalancerP2P front-end will share a similar...,445
1902,472,"[link]([link]) | 287,507.81357388 | | Main...",483
831,210,Ethereum >mainnet: [link] 'enableRecoveryMode'...,557
2205,548,"Aave V2| amBAL| $5.51| 1,125.41| $ 6,201.01 |...",558


In [174]:
long_speech_split_lengths = long_speech_split['Sections'].str.len().to_numpy()
long_speech_split_lengths.sort()
print(f'Amount of symbols: {long_speech_split_lengths[::-1][:10]} ... ')

Amount of symbols: [1873 1610 1354 1282 1240 1088 1078 1057 1053 1024] ... 


## Summarize

In [175]:
!nvidia-smi -L

GPU 0: Tesla V100-SXM2-16GB (UUID: GPU-87686a2c-6ea6-f273-20e1-360ce59c6aee)


In [169]:
nlp_summarizer = pipeline("summarization", model="facebook/bart-large-cnn", device=0)

In [188]:
%%time

corpus_long = list(long_speech_split['Sections'].values)
summaries_list = []
summaries_list_temp = []



for i, chunk in enumerate(corpus_long[:]):
  chunk_token_len = len(tokenizer.tokenize(chunk))
  max_length = int(chunk_token_len/2) if chunk_token_len > 10 else chunk_token_len
  min_length = int(chunk_token_len/10)
  summary = nlp_summarizer(chunk, max_length=max_length, min_length=min_length, do_sample=False)
  print(f'\n{i} Chunk:\n', chunk, '\nSummary:\n', summary, f'\nmax={max_length}, min={min_length}, len={len(chunk)}, tok={len(tokenizer.tokenize(chunk))}')
  summaries_list.append(summary)


0 Chunk:
 Simple Summary. This proposal calls for the integration of [Balancer Boosted Pools]([link]) into the 1inch Aggregation Protocol. Abstract. This proposal aims to grow the kinds of liquidity sources compatible with the 1inch Aggregation Protocol by integrating Balancer Boosted Pools. To fund this development, a BAL grant from Balancer DAO and Balancer DAO will be awarded to 1inch Labs upon the successful completion of the integration. Specifically, the following. 1. 1inch Labs shall integrate the Balancer Boosted Pools into the 1inch smart contracts, the 1inch dApp, and the 1inch API. 2. The integration shall be live on the Ethereum main-net no later than 3 weeks after this proposal passes the Phase-4 snapshot vote. 3. If both conditions 1 and 2 are met, Balancer DAO and Balancer DAO will transfer 20k BAL or the equivalent in stablecoins (as determined by Balancer DAO) to 1inch Labs. Motivation. Balancer Boosted Pools are a new pool type that have been gaining traction from ma

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1081: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Выходные данные были обрезаны до нескольких последних строк (5000).

1927 Chunk:
 Agave was built by 1Hive to bring Defi to what is now Gnosis Chain for lower fees to participate overall. Gnosis took over xDAI chain and will now be the core token for the network in terms of governance and validation. USDC is the dollar backed stablecoin issued by Circle. DAI is the decentralized overcollateralized stablecoin by MakerDAO. USDT is the dollar backed stablecoin issued by Tether. Motivation: > This Agave boosted pool couples the boosted USD and GNO tokens to facilitate a strong liquidity base on Balancer with Gnosis. By deploying on Gnosis Chain, logically supporting GNO as a trading exemplifies Balancers commitment to the chain. The boosted APR yield to this pool that will persist for users organically and kickstart a flywheel for incentives to drive liquidity depth on Gnosis Chain. By expanding to a new chain, getting boosted pools off the ground first thing sets a precedent for net posit

## Process summirized results

In [190]:
summaries = []
for summary in summaries_list:
  summaries.append(str(summary)[19:-3])

In [194]:
long_speech_split['Summary']=summaries
long_speech_split

,level_0,Sections,SectionsLength,Summary
0,0,Simple Summary. This proposal calls for the in...,228,Proposal aims to grow the kinds of liquidity s...
1,0,These pools were previously discussed with the...,212,Balancer seeks to fund this integration via a ...
2,0,In the near future we expect to see meta pools...,257,In the near future we expect to see meta pools...
3,0,Support for the mentioned pools will unlock $3...,203,Support for the mentioned pools will unlock $3...
4,0,The technical specifics of this integration wi...,66,Security review and audit should be performed ...
...,...,...,...,...
2755,682,[[1IP-14] Update 1inch DAO Voting Process to I...,240,This proposal marks the full deprecation of st...
2756,682,"Currently, all v1INCH, no matter where it is a...",254,v1INCH will have more voting power if there is...
2757,682,New scheme = 17K votes Example 3 Address: [hex...,262,The [1inch Network Snapshot space] and the[1in...
2758,682,Change the proposal validation strategy to req...,247,Change the proposal validation strategy to req...


In [ ]:
df_summirized = pd.DataFrame()
df_summirized['seniorDescription']=df['seniorDescription']
df_summirized['HF_extract_summarization'] = long_speech_split[['level_0','Sections','Summary']].groupby(['level_0'])['Summary'].apply(list).str.join(' ')
df_summirized

In [ ]:
df['extractiveSummarization'] = df_summirized['HF_extract_summarization']
df.drop(['seniorSplit'], axis=1)

In [197]:
df.to_csv('../data/BART.csv', index=False)